# Preparation and Reading Input Data

In [ ]:
%matplotlib inline
import sys
import numpy as np
import random
import os, time, sys, datetime
import sklearn.metrics.pairwise
import scipy.spatial.distance
from scipy.spatial.distance import cdist
import dask
import dask.array as da
import dask.multiprocessing
from dask.diagnostics import ProgressBar
from dask.diagnostics import ResourceProfiler
from dask.array.core import map_blocks
from multiprocessing.pool import ThreadPool

In [ ]:
RESULT_DIR="results"
RESULT_FILE_PREFIX="pair-distance-"
HEADER_CSV="Scenario, Type, Time"
OUT_DIR="../npy_stack"


FILENAMES=["../132k_dataset/atom_pos_132K.npy", "../145K_dataset/atom_pos_145K.npy", 
          "../300K_dataset/atom_pos_291K.npy", '../840K_dataset/atom_pos_839K.npy']

scenario = FILENAMES[0]

## Preprate npy stacks for Dask

In [ ]:
CHUNKSIZE=4096

for i in FILENAMES:
    print i
    atoms = np.load(i)
    a_da = da.from_array(atoms, chunks=(CHUNKSIZE,3))
    out_file=os.path.join(OUT_DIR, os.path.basename(i)+"_"+str(CHUNKSIZE))
    try:
        os.makedirs(out_file)
    except:
        pass
    da.to_npy_stack(out_file, a_da)

In [ ]:
!rm

In [ ]:
!ls ../npy_stack/

# Load Data Numpy

In [ ]:
start = time.time()
atoms = np.load(scenario)
atoms.shape
end = time.time()

try:
    os.mkdir(RESULT_DIR)
except:
    pass        

results=[]
d =datetime.datetime.now()
result_filename = RESULT_FILE_PREFIX + d.strftime("%Y%m%d-%H%M%S") + ".csv"
f = open(os.path.join(RESULT_DIR, result_filename), "w")
f.write(HEADER_CSV+ "\n")
result_line = scenario + ",Read," + str(end-start)
f.write(result_line)
f.flush()
print result_line

In [ ]:
a=atoms[:]
a.shape

# Distance Calculation - Single Core

## Giannis

In [ ]:
%%time

def get_distance(Atom1, Atom2):
    # Calculate Euclidean distance. 1-D and 3-D in the future
    return np.sqrt(sum((Atom1 - Atom2) ** 2))

def n_dim_input_to_numpy_array(temp):
    temp = temp.split(',')
    temp = map(float,temp)
    return np.asfarray(temp)
    
    
# the difference is that in the Cus compute data that are in main diagonal compute half of the elements 
# because table is symmetric, so the second loop can be half in the first case 

WINDOW_SIZE=a.shape[0]
reading_start_point_i = 0
j_dim = 0
cutoff = 15
print WINDOW_SIZE


def distance_giannis():
    distances=np.empty((WINDOW_SIZE, WINDOW_SIZE), dtype='bool')    
    for i in range(0, WINDOW_SIZE):
        for j in range(i+1,WINDOW_SIZE):
            dist = get_distance(a[i],a[j])  
            if dist<=cutoff:
                distances[i][j]=True 
            else:
                distances[i][j]=False
    return distances

## Python Plain NumPy

In [ ]:
%%time
dist_sq = np.sqrt(np.sum((a[:,np.newaxis,:] - a[np.newaxis,:,:]) ** 2, axis=-1))

## Scikit

In [ ]:
%%time
dist_sq = cdist(a, a, 'euclidean')

In [ ]:
print dist_sq.shape

# Dask

In [ ]:
a_dask=da.from_array(a, chunks=(2048,3))

## Dask Threads

In [ ]:
def benchmark_dask(filename, cutoff=15, number_threads=40):
    results = []
    start = time.time()
    a_dask=da.from_npy_stack(filename)
    end_read = time.time()
    diff=a_dask[:, np.newaxis, :] - a_dask[np.newaxis, :, :]
    res=da.sqrt(da.sum((diff) ** 2, axis=-1))>cutoff
    with ProgressBar():
        with dask.set_options(get=dask.multiprocessing.get):
            out = res.compute()
            print type(out)
            end_compute = time.time()
            np.save(os.path.basename(filename)+"_out.npy", out)
            #da.to_npy_stack("out.npy", out)
            end_out_write = time.time()
            results.append("%s,dask,thread-40,read_file, %.4f"%(filename, end_read-start))
            results.append("%s,dask,thread-40,compute, %.4f"%(filename, end_compute-end_read))
            results.append("%s,dask,thread-40,write_file, %.4f"%(filename, end_out_write-end_compute))
            results.append("%s,dask,thread-40,total, %.4f"%(filename, end_out_write-start))
            print("\n".join(results))

## Dask Distributed

In [ ]:
from distributed import Client, progress

hostname = 'xxx:8786'

def benchmark_dask_distributed(filename, cutoff=15, number_threads=40):
    client = Client(hostname)
    results = []
    start = time.time()
    a_dask=da.from_npy_stack(filename)
    end_read = time.time()
    diff=a_dask[:, np.newaxis, :] - a_dask[np.newaxis, :, :]
    res=da.sqrt(da.sum((diff) ** 2, axis=-1))>cutoff 
    with ProgressBar():
        with dask.set_options(get=client.get):
            out = res.compute()
            end_compute = time.time()
            print "end compute"
            outfile = os.path.join("/data/tmp", os.path.basename(filename)+"_out.npy")
            np.save(outfile, out)
            #da.to_npy_stack("out.npy", out)
            end_out_write = time.time()            
            os.remove(outfile)
            results.append("%s,dask,thread-distributed,read_file, %.4f"%(filename, end_read-start))
            results.append("%s,dask,thread-distributed,compute, %.4f"%(filename, end_compute-end_read))
            results.append("%s,dask,thread-distributed,write_file, %.4f"%(filename, end_out_write-end_compute))
            results.append("%s,dask,thread-distributed,total, %.4f"%(filename, end_out_write-start))
            print("\n".join(results))
    #client.shutdown()

In [ ]:
dask_scenarios = [os.path.abspath(os.path.join(OUT_DIR, i)) for i in os.listdir(OUT_DIR)]
for s in dask_scenarios:
    if "_4096" in s:
        print "Process: %s"%s
        benchmark_dask(s)

In [ ]:
!ls ../npy_stack/atom_pos_132K.npy

## Map Blocks
(still exploratory)

In [ ]:
%%time

f = map_blocks(lambda a, b: cdist(a,b, 'euclidean'), a_dask, a_dask)
res=f.compute()

In [ ]:
number_points = a_dask.shape[0]
distance_matrix=da.ones((number_points, number_points), chunks=(128, 128))

In [ ]:
dist=map_blocks(lambda a: a[0, 1], distance_matrix)

In [ ]:
res=dist.compute()

In [ ]:
print res

# MXNet

In [ ]:
import mxnet as mx

In [ ]:
%%time
a_mx=mx.nd.array(a)
diff=(mx.ndarray.power(a_mx.reshape((1, a_mx.shape[0], a_mx.shape[1])) - a_mx.reshape((a_mx.shape[0], 1, a_mx.shape[1])), 2))
dist=mx.ndarray.sqrt(mx.ndarray.sum_axis(diff, axis=2)).asnumpy()

In [ ]:
dist

# Tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

# Benchmark All

In [ ]:
RESULT_DIR="results"
RESULT_FILE_PREFIX="mdanalysis-distance-spark-"
HEADER_CSV="Scenario, NumberAtoms, NumberExecutors, Time"

try:
    os.mkdir(RESULT_DIR)
except:
    pass        

results=[]
d =datetime.datetime.now()
result_filename = RESULT_FILE_PREFIX + d.strftime("%Y%m%d-%H%M%S") + ".csv"
f = open(os.path.join(RESULT_DIR, result_filename), "w")
f.write(HEADER_CSV+ "\n")


for i in range(10):
    for file_name in files:
        print "Process: " + file_name
        coord = np.loadtxt(file_name, dtype='float32')
        for i in NUMBER_EXECUTORS_SCENARIOS:
            #result=benchmark_spark(coord, i)
            result=benchmark_mdanalysis_dense(coord, i)
            #result=benchmark__tf_batch(coord, 500)
            results.append(result)
            f.write(result + "\n")
            f.flush()            
        del coord
        gc.collect()

f.close()
print("Finished run")

# Testing and Playing around with Numpy Broadcasting

In [ ]:
a_np = np.arange(15).reshape(5,3)
a_np

In [ ]:
a_np.shape[-1]

In [ ]:
x=a_np[:, np.newaxis, :]
y=a_np[np.newaxis,:,:]

In [ ]:
y.shape

In [ ]:
x.shape

In [ ]:
(x-y).shape